# Drive mount

In [ ]:
#Colab에 파일 업로드 시 직접 업로드보다는 구글 드라이브 연동을 추천합니다.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Import

In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install datasets
!pip install transformers[torch] -U
!pip install peft
!pip install bitsandbytes
!pip install trl
!pip install nlpaug
!pip install textaugment
!pip install contractions

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm import tqdm

import nlpaug.augmenter.word as naw #데이터 증강을 위한 라이브러리
from nlpaug.util import Action
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import unicodedata
import contractions
from nltk.corpus import stopwords

import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
     BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import load_dataset, Dataset, DatasetDict
from accelerate import Accelerator


# 데이터를 Colab 서버 내에 세팅

In [ ]:
# open.zip 파일을 업로드한 경로를 확인하여 적용
!unzip  "/content/drive/MyDrive/open.zip"

Archive:  /content/drive/MyDrive/open.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


#데이터 증강

# Load Validation / Test dataset

In [ ]:
file_path = '/content/train.csv'
train_data = pd.read_csv(file_path)



# 데이터 전처리: 중복 제거, 결측값 처리, 텍스트 정규화
train_data.drop_duplicates(inplace=True)
train_data.dropna(subset=['context', 'question', 'answer'], inplace=True)

def clean_text(text):
    text = text.lower()  # 소문자 변환
    text = unicodedata.normalize("NFKD", text) # 유니코드 정규화 (발음 구별 기호 등 제거)
    text = contractions.fix(text) # 축약어 복원
    text = re.sub(r'\s+', ' ', text)  # 여러 공백을 하나의 공백으로 변환
    text = re.sub(f"[{string.punctuation}]", '', text)  # 구두점 제거
    return text

train_data['context'] = train_data['context'].apply(clean_text)
train_data['question'] = train_data['question'].apply(clean_text)
train_data['answer'] = train_data['answer'].apply(clean_text)


train_data = train_data.sample(frac=1).reset_index(drop=True)

#여기도 개선 가능!
val_data=train_data[:10]
train_data = train_data[2000:5001]

val_label_df = val_data[['question', 'answer']]

train_data.head(5)

,id,context,question,answer
2000,TRAIN_30483,한국소비자원원장 이희숙은 fta체결국으...,하위 5개 품목의 평균 후생지수,65점
2001,TRAIN_27068,코로나19 사태의 장기화로 ‘홈술’집에서 술 ...,페트병 묶음 소주 매출 증가율,41
2002,TRAIN_20735,전라북도경제통상진흥원원장 조지훈으...,전북이 생산한 고품질의 쌀을 싸게 살 ...,대한민국 농할갑시다
2003,TRAIN_33278,현대차그룹은 10일 전기차에서 회수한 배터...,ess의 전력 저장 용량은,2mwh
2004,TRAIN_26260,충북 지역이 우리나라 전체 무역흑자 중 3...,청주시가 무역으로 얼마의 이익을 달성했어,163억 9000만달러


#fine-tuning data 제작

In [ ]:
# 변환 함수 정의
def transform_row(row):
    context = row['context']
    question = row['question']
    answer = row['answer']
    return f"<s>[INST] 너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해. Context: {context} Question: {question}[/INST] {answer} </s>"

In [ ]:
train_data['text'] = train_data.apply(transform_row, axis=1)

In [ ]:
train_data = Dataset.from_pandas(train_data[['text']])

In [ ]:
print(train_data)

Dataset({
    features: ['text'],
    num_rows: 3001
})


In [ ]:
print(train_data[0])

{'text': '<s>[INST] 너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해. Context: 한국소비자원원장 이희숙은 fta체결국으로부터 수입되는 39개 품목에 대한 지난해 소비자 후생지수를 측정한 결과 코로나19가 국내 소비자의 후생에 크게 영향을 준 것으로 나타났다 코로나19로 집에 머무는 시간이 많아지면서 종합비타민 커피머신 와인 향수 등 홈코노미 관련 수입소비재의 소비자 후생지수가 상위를 차지했다 특히 종합비타민이 총 39개 수입소비재 품목 가운데 소비자 후생지수 순위 1위 품목으로 나타나 전염병의 세계적 대유행에 따라 건강에 대한 관심이 종합비타민의 소비 증가로 이어진 것으로 보인다 ‘홈파티’ᆞ ‘혼술’ 문화 스마트오더모바일 주문 후 매장에서 수령 편의점 판매증가 등 국내 소비환경이 크게 변화된 와인은 수입액3억3000만이 지난 2016년1억9000만에 비해 724가 증가했으며 소비자 후생지수도 1517점으로 2019년 1106점 보다 372가 상승해 가장 높은 지수상승률을 기록했다 ᄆ

In [ ]:
# 현재 사용중인 GPU의 CUDA 연산 능력을 확인한다.
# 8이상이면 고성능 GPU 로 판단한다.
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    # 고성능 Attention인 flash attention 2 을 사용
    attn_implementation = "flash_attention_2"
    # 데이터 타입을 bfloat16으로 설정해준다.
    # bfloat16은 메모리 사용량을 줄이면서도 계산의 정확성을 유지할 수 있는 데이터 타입이다.
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

In [ ]:
# QLoRA config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,	# 모델 가중치를 4비트로 로드
    bnb_4bit_quant_type="nf4",	# 양자화 유형으로는 “nf4”를 사용한다.
    bnb_4bit_compute_dtype=torch_dtype,	# 양자화를 위한 컴퓨팅 타입은 직전에 정의 했던 torch_dtype으로 지정 해준다.
    bnb_4bit_use_double_quant=False,	# 이중 양자화는 사용하지 않는다.
)

# Model Load

In [ ]:
# llama-2-ko-7b 모델 로드
base_model = "beomi/llama-2-ko-7b"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto"
    #device_map={"": 0}	# 0번째 gpu 에 할당
)
# 모델의 캐시 기능을 비활성화 한다. 캐시는 이전 계산 결과를 저장하기 때문에 추론 속도를 높이는 역할을 한다. 그러나 메모리 사용량을 증가시킬 수 있기 때문에, 메모리부족 문제가 발생하지 않도록 하기 위해 비활성화 해주는 것이 좋다.
model.config.use_cache = False
# 모델의 텐서 병렬화(Tensor Parallelism) 설정을 1로 지정한다. 설정값 1은 단일 GPU에서 실행되도록 설정 해주는 의미이다.
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/919M [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

model-00013-of-00015.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/742M [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [ ]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
              base_model,
              trust_remote_code=True)
# 시퀀스 길이를 맞추기 위해 문장 끝에 eos_token를 사용
tokenizer.pad_token = tokenizer.eos_token
# 패딩 토큰을 시퀀스의 어느 쪽에 추가할지 설정
tokenizer.padding_side = "right"

new_model = "beomi/llama-2-ko-7b-finetuned"

tokenizer_config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,	# LoRA의 스케일링 계수를 설정 한다. 값이 클 수록 학습 속도가 빨라질 수 있지만, 너무 크게 되면 모델이 불안정해질 수 있다.
    lora_dropout=0.1,	#  과적합을 방지하기 위한 드롭아웃 확률을 설정한다. 여기서는 10%(0.1)의 드롭아웃 확률을 사용하여 모델의 일반화 성능을 향상시킨다.
    r=64,	# LoRA 어댑터 행렬의 Rank를 나타낸다. 랭크가 높을수록 모델의 표현 능력은 향상되지만, 메모리 사용량과 학습 시간이 증가한다. 일반적으로 4, 8, 16, 32, 64 등의 값을 사용한다.
    bias="none",	# LoRA 어댑터 행렬에 대한 편향을 추가할지 여부를 결정한다. “none”옵션을 사용하여 편향을 사용하지 않는다.
    task_type="CAUSAL_LM",	# LoRA가 적용될 작업 유형을 설정한다. CAUSAL_LM은 Causal Language Modeling 작업을 의미한다. 이는 특히 GPT 같은 텍스트 생성 모델에 주로 사용된다.
)

In [ ]:
# 만약 CUDA memory error 뜨면 이거 사용! to 진노스케

training_params = TrainingArguments(
    output_dir="results",
    num_train_epochs = 1, #epoch는 1로 설정
    max_steps=5000, #max_steps을 5000으로 설정
    per_device_train_batch_size=1,
    gradient_accumulation_steps=3,
    optim="paged_adamw_8bit",
    warmup_steps=0,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=100,
    push_to_hub=False,
    report_to='none',
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=512,	# 256, 512 등으로 수정할 수 있음.
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/3001 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Step,Training Loss
100,2.075800
200,1.978500
300,1.960800
400,1.964200
500,1.925600
600,1.904300
700,1.933200
800,1.919800
900,1.939000
1000,1.903000


In [ ]:
trainer.save_model(new_model)

In [ ]:
# 모델 가중치 저장
model.save_pretrained("model_checkpoint")
tokenizer.save_pretrained("model_checkpoint")

# Define F1 score

In [ ]:
def normalize_answer(s):
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text)
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        '''연속된 공백일 경우 하나의 공백으로 대체'''
        return ' '.join(text.split())

    def remove_punc(text):
        '''구두점 제거'''
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        '''소문자 전환'''
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()

    # 문자 단위로 f1-score를 계산 합니다.
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)

    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)

    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

def evaluate(ground_truth_df, predictions_df):
    predictions = dict(zip(predictions_df['question'], predictions_df['answer']))
    f1 = exact_match = total = 0

    for index, row in ground_truth_df.iterrows():
        question_text = row['question']
        ground_truths = row['answer']
        total += 1
        if question_text not in predictions:
            continue
        prediction = predictions[question_text]
        f1 = f1 + f1_score(prediction, ground_truths)

    f1 = 100.0 * f1 / total
    return {'f1': f1}


# Validation
- 로드한 모델에 대해 간단한 프롬프트를 입력하여 성능 검증

In [ ]:
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
# 모델로 추론 후, 전처리를 수행한 뒤, 완성된 정답으로 반환합니다.
def generate_response(question_prompt):
    # 생성할 최대 토큰 수와, 답변 생성 수, 패딩 토큰의 idx를 지정하여 모델 파이프 라인을 설정하고, 답변을 생성합니다.
    response = qa_pipeline(question_prompt, max_new_tokens=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)[0]['generated_text']
    if "Answer:" in response:
            # Answer: 이후에 생성된 토큰 들만을 답변으로 사용합니다.
            response = response.split("Answer:", 1)[1][:20]

            # 토큰 반복 생성 및 노이즈 토큰 관련 처리
            if "Que" in response:
                response = response.split("Que", 1)[0]
            if "⊙" in response:
                response = response.split("⊙", 1)[0]
            if "Con" in response:
                response = response.split("Con", 1)[0]

    return response

In [ ]:
#Google Colab T4 GPU 기준 10개 샘플 답안 생성에 1분 소요
predict_dict = {}
count = 0

for index, row in val_data.iterrows():
    try:
        context = row['context']
        question = row['question']


        if context is not None and question is not None:
            question_prompt = f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
                                Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해. \
                                Context: {context} Question: {question}\n Answer:"

            answer = generate_response(question_prompt)
            predict_dict[question] = answer
        else:
            predict_dict[question] = 'Invalid question or context'

        print("Answer for question:", question, ":", predict_dict[question])
        count += 1
        print("Processed count:", count)
    except Exception as e:
        print(f"Error processing question {e}")



In [ ]:
val_inf_df = pd.DataFrame(list(predict_dict.items()), columns=['question', 'answer'])
val_inf_df.head()

In [ ]:
# f1-score 를 출력합니다.
results = evaluate(val_inf_df, val_label_df)
print(results)

# Inference

In [ ]:
file_path = '/content/test.csv'
test_data = pd.read_csv(file_path)

In [ ]:
# 모델 추론
submission_dict = {}
count = 0

for index, row in test_data.iterrows():
    try:
        context = row['context']
        question = row['question']
        id = row['id']

        if context is not None and question is not None:
            question_prompt = f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
                                Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해. \
                                Context: {context} Question: {question}\n Answer:"

            answer = generate_response(question_prompt)
            submission_dict[id] = answer
        else:
            submission_dict[id] = 'Invalid question or context'

        print("Answer for question:", question, ":", submission_dict[id])
        count += 1
        print("Processed count:", count)

    except Exception as e:
        print(f"Error processing question {e}")

# Submission

In [ ]:
# 제출
df = pd.DataFrame(list(submission_dict.items()), columns=['id', 'answer'])
df.to_csv( '/content/submission.csv', index=False)